<a href="https://colab.research.google.com/github/RishabhNegi1/Chat-Bot/blob/main/Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install tensorflow keras nltk numpy pickle transformers


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [11]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
from google.colab import files
files.upload()  # Use this to upload chatbot_Application_model.h5, intents.json, labels.pkl, words.pkl


Saving chatbot_Application_model.h5 to chatbot_Application_model (1).h5
Saving intents.json to intents (1).json
Saving labels.pkl to labels (1).pkl
Saving words.pkl to words (1).pkl


{'chatbot_Application_model (1).h5': b'\x89HDF\r\n\x1a\n\x00\x00\x00\x00\x00\x08\x08\x00\x04\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\x90\xf1\x02\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00`\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x88\x00\x00\x00\x00\x00\x00\x00\xa8\x02\x00\x00\x00\x00\x00\x00\x01\x00\x07\x00\x01\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x10\x00\x10\x00\x00\x00\x00\x00 \x03\x00\x00\x00\x00\x00\x00h\x01\x00\x00\x00\x00\x00\x00TREE\x00\x00\x01\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
# Step 4: Import libraries and load the saved model, words, labels, and intents file
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import json
import random
import pickle
from keras.models import load_model
from transformers import AutoModelForCausalLM, AutoTokenizer

# Lemmatizer to reduce words to their root form
lemmatizer = WordNetLemmatizer()

# Load the trained model, intents dataset, words, and labels
model = load_model('chatbot_Application_model.h5')
intents = json.loads(open('intents.json').read())  # Load intents
words = pickle.load(open('words.pkl', 'rb'))  # Load words
labels = pickle.load(open('labels.pkl', 'rb'))  # Load labels

# Step 5: Enhance chatbot with a pre-trained transformer model (DialoGPT)
# Loading the pre-trained DialoGPT model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
dialoGPT_model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Step 6: Define helper functions for bag_of_words, prediction, and response generation

def bag_of_words(s, words):
    """Convert input sentence into bag of words format"""
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]

    for word in s_words:
        for i, w in enumerate(words):
            if w == word:
                bag[i] = 1
    return np.array(bag)

def predict_intent(s, model):
    """Predict the intent of the user based on input"""
    bow = bag_of_words(s, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []
    for r in results:
        return_list.append({"intent": labels[r[0]], "probability": str(r[1])})

    return return_list

def get_response(ints, intents):
    """Return response based on predicted intent"""
    tag = ints[0]['intent']
    list_of_intents = intents['intents']  # Here intents is the correct dictionary from the file
    for i in list_of_intents:
        if i['tag'] == tag:
            return random.choice(i['responses'])

# DialoGPT Response
def generate_transformer_response(input_text):
    """Generate response using DialoGPT"""
    inputs = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')
    response_ids = dialoGPT_model.generate(inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response_text = tokenizer.decode(response_ids[:, inputs.shape[-1]:][0], skip_special_tokens=True)
    return response_text

# Step 7: Combine both response methods (classification and transformer-based)
def chatbot_response(msg):
    """Choose between classification-based or transformer-based response"""
    predicted_intents = predict_intent(msg, model)

    # Check confidence threshold for intent classification, fallback to DialoGPT if needed
    if predicted_intents and float(predicted_intents[0]['probability']) > 0.75:
        return get_response(predicted_intents, intents)  # Pass correct intents object here
    else:
        # If confidence is low, use the transformer-based model for a more flexible response
        return generate_transformer_response(msg)

# Step 9: Chat with the bot
def chat():
    """Start the chatbot for user interaction"""
    print("Start chatting with the bot (type 'quit' to stop)!")
    while True:
        msg = input("You: ")
        if msg.lower() == "quit":
            break
        print(f"Bot: {chatbot_response(msg)}")

# Step 10: Start the chatbot
chat()


Start chatting with the bot (type 'quit' to stop)!
You: Hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Bot: Hi there, how can I help?
You: Help me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Bot: I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies
You: Hospitals
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: Please provide hospital name or location
You: USA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: Hello, thanks for asking
You: Hey
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: Good to see you again
You: Whats your name
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Good to see you again
You: Hep me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot: I'm sorry, I don't speak German.
You: Help me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies
You: Drugs 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Navigating to Adverse drug reaction module
You: After that?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: My pleasure
